In [1]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import matthews_corrcoef, accuracy_score, f1_score, balanced_accuracy_score, precision_recall_fscore_support, classification_report
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np

import os
os.environ["WANDB_PROJECT"] = "offline"

training_directory ='fewshot'
#modname = "training_base/best_checkpoint_augmented"
modname = "training_large/latest_checkpoint"
modname = 'mlburnham/Political_DEBATE_large_v1.0'
# instantiate model
model = AutoModelForSequenceClassification.from_pretrained(modname, num_labels = 2, ignore_mismatched_sizes=True)

def metrics(df, preds, group_by=None):
    true_col = 'entailment'
    
    def get_metrics(y_true, y_pred):
        return {
            'MCC': matthews_corrcoef(y_true, y_pred),
            'Accuracy': accuracy_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred, average='weighted')
        }
    
    results = []
    
    if group_by not in ['dataset', 'task']:
        for col in preds:
            metrics = get_metrics(df[true_col], df[col])
            metrics['Column'] = col
            results.append(metrics)
    else:
        for col in preds:
            for group_name, group in df.groupby(group_by):
                metrics = get_metrics(group[true_col], group[col])
                metrics['Column'] = col
                metrics[group_by.capitalize()] = group_name
                results.append(metrics)
    
    results_df = pd.DataFrame(results)
    
    if group_by in ['dataset', 'task']:
        return results_df.set_index(['Column', group_by.capitalize()])
    else:
        return results_df.set_index('Column')

def truncate(text):
    words = text.split()
    if len(words) > 450:
        return " ".join(words[:450])
    return text


def tokenize_function(docs):
    return tokenizer(docs['premise'], docs['hypothesis'], padding = 'max_length', truncation = True)


def compute_metrics_standard(eval_pred, label_text_alphabetical=list(model.config.id2label.values())):
    labels = eval_pred.label_ids
    pred_logits = eval_pred.predictions
    preds_max = np.argmax(pred_logits, axis=1)

    # metrics
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds_max, average='macro') 
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds_max, average='micro')
    acc_balanced = balanced_accuracy_score(labels, preds_max)
    acc_not_balanced = accuracy_score(labels, preds_max)
    mcc = matthews_corrcoef(labels, preds_max)

    metrics = {'MCC': mcc,
            'f1_macro': f1_macro,
            'f1_micro': f1_micro,
            'accuracy_balanced': acc_balanced,
            'accuracy': acc_not_balanced,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'precision_micro': precision_micro,
            'recall_micro': recall_micro,
            }
    print("Aggregate metrics: ", {key: metrics[key] for key in metrics if key not in ["label_gold_raw", "label_predicted_raw"]} )
    print("Detailed metrics: ", classification_report(
        labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
        target_names=label_text_alphabetical, sample_weight=None,
        digits=2, output_dict=True, zero_division='warn'),
    "\n")

    return metrics

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Freedom and Rights

In [2]:
fr = pd.read_csv('freedom_test.csv')
fr = fr[~fr['text'].isna()]
fr['text'] = fr['text'].astype(str)
fr['hypothesis'] = 'This text is about freedom and rights.'
fr.rename({'text':'premise', 'freedom_and_rights':'entailment'}, inplace = True, axis = 1)
fr.drop_duplicates('premise', inplace = True)
fr['entailment'].replace({1:0, 0:1}, inplace = True)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\886333745.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fr['entailment'].replace({1:0, 0:1}, inplace = True)


### Few Shot

In [3]:
training_args = TrainingArguments(output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    lr_scheduler_type= "linear",
    group_by_length=False,
    learning_rate = 9e-6,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 1, 
    num_train_epochs=5,
    warmup_ratio=0.06,  
    weight_decay=0.01, 
    fp16=True,   
    fp16_full_eval=True,
    eval_strategy="no",
    seed=1,
    save_strategy="no",
    dataloader_num_workers = 12,
)

tokenizer = AutoTokenizer.from_pretrained(modname)

In [4]:
%%time
# Define a function to initialize the modelin the trainer. This will make results reproducible
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modname, num_labels = 2, ignore_mismatched_sizes=True)

# Define the number of samples (shots) and random seeds to use
shots = [10, 25, 50, 100]
seeds = range(1,11)

# Initialize lists to store results
mcc_list = []
acc_list = []
shots_list = []

# Iterate through different shot sizes
for shot in shots:
    # Iterate through different random seeds
    for seed in seeds:
        # Sample training data based on current shot size and seed
        train = fr.sample(shot, random_state = seed)
        # Create validation set with remaining instances
        val = fr[~fr.index.isin(train.index)]
        
        # Create a DatasetDict with train and validation splits
        ds = DatasetDict({'train': Dataset.from_pandas(train, preserve_index=False), 'validation':Dataset.from_pandas(val, preserve_index=False)})
        # Tokenize the dataset
        dstok = ds.map(tokenize_function, batched = True)
        # Rename 'entailment' column to 'label'
        dstok = dstok.rename_columns({'entailment':'label'})
        # Define label mapping
        id2label = {0: "entailment", 1: "not_entailment"}
        
        # Initialize the Trainer
        trainer = Trainer(
            model_init = model_init,
            tokenizer=tokenizer,
            args=training_args,
            train_dataset=dstok['train'],
            eval_dataset=dstok['validation'],
            compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=list(model.config.id2label.values()))
        )
        
        # Train the model
        trainer.train()
        # Make predictions on validation set
        res = trainer.predict(dstok['validation'])
        preds = np.argmax(res.predictions, axis=-1)
        
        # Calculate Matthews Correlation Coefficient
        mcc_res = matthews_corrcoef(val['entailment'], preds)
        mcc_list.append(mcc_res)
        # Calculate Accuracy
        acc_res = accuracy_score(val['entailment'], preds)
        acc_list.append(acc_res)
        # Store the current shot size
        shots_list.append(shot)
    
    # Print progress
    print(shot)

Map: 100%|██████████| 7620/7620 [00:01<00:00, 5828.75 examples/s]
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mlburnham. Use `wandb login --relogin` to force relogin


Step,Training Loss


Aggregate metrics:  {'MCC': 0.7844197233481313, 'f1_macro': 0.8902844007009825, 'f1_micro': 0.9056430446194226, 'accuracy_balanced': 0.8786685590360639, 'accuracy': 0.9056430446194226, 'precision_macro': 0.9062354054056527, 'recall_macro': 0.8786685590360639, 'precision_micro': 0.9056430446194226, 'recall_micro': 0.9056430446194226}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\1171338601.py:86: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.9076647243388615, 'recall': 0.7978723404255319, 'f1-score': 0.8492346403858251, 'support': 2538.0}, 'not_entailment': {'precision': 0.9048060864724439, 'recall': 0.9594647776465959, 'f1-score': 0.9313341610161399, 'support': 5082.0}, 'accuracy': 0.9056430446194226, 'macro avg': {'precision': 0.9062354054056527, 'recall': 0.8786685590360639, 'f1-score': 0.8902844007009825, 'support': 7620.0}, 'weighted avg': {'precision': 0.9057582154625972, 'recall': 0.9056430446194226, 'f1-score': 0.9039892025699799, 'support': 7620.0}} 



...

Map: 100%|██████████| 7530/7530 [00:01<00:00, 5355.38 examples/s]
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Aggregate metrics:  {'MCC': 0.7888637921804589, 'f1_macro': 0.8899049767610823, 'f1_micro': 0.9069057104913678, 'accuracy_balanced': 0.8726837842055779, 'accuracy': 0.9069057104913678, 'precision_macro': 0.9174491277772236, 'recall_macro': 0.8726837842055779, 'precision_micro': 0.9069057104913678, 'recall_micro': 0.9069057104913678}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\1171338601.py:86: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.9406903257170637, 'recall': 0.7696897374701671, 'f1-score': 0.8466418726755633, 'support': 2514.0}, 'not_entailment': {'precision': 0.8942079298373835, 'recall': 0.9756778309409888, 'f1-score': 0.9331680808466012, 'support': 5016.0}, 'accuracy': 0.9069057104913678, 'macro avg': {'precision': 0.9174491277772236, 'recall': 0.8726837842055779, 'f1-score': 0.8899049767610823, 'support': 7530.0}, 'weighted avg': {'precision': 0.9097267536410377, 'recall': 0.9069057104913678, 'f1-score': 0.9042800479990595, 'support': 7530.0}} 

100
CPU times: total: 1h 42min 38s
Wall time: 4h 45min 9s


In [5]:
res_fr = pd.DataFrame({'n':shots_list, 'mcc':mcc_list})
res_fr['accuracy'] = acc_list
res_fr.groupby('n').mean()

,mcc,accuracy
n,,
10,0.766817,0.894843
25,0.756960,0.893465
50,0.782889,0.904472
100,0.806834,0.914635


### Zero Shot

In [6]:
pipe = pipeline("zero-shot-classification", model = modname, device = 0, batch_size = 32)
res = pipe(list(fr['premise'].str.lower()), ['freedom and rights except voting.'], hypothesis_template = 'This text is about {}', multi_label = False)
labels = [round(label['scores'][0], 0) for label in res]
fr['0_shot'] = labels
fr['0_shot'].replace({0:1, 1:0}, inplace = True)

zs_fr = pd.DataFrame({'n':0, 'mcc':matthews_corrcoef(fr['entailment'], fr['0_shot']), 'accuracy':accuracy_score(fr['entailment'], fr['0_shot'])}, index = [0])
res_fr = pd.concat([res_fr, zs_fr], axis = 0)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\3963122802.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fr['0_shot'].replace({0:1, 1:0}, inplace = True)


In [7]:
res_fr.to_csv('motn_fewshot_large2.csv', index = False)

# Covid

In [8]:
df = pd.read_csv('covid_tweets_labeled.csv')
df = df[['text', 'non_comp']]
df['hypothesis'] = 'The author of this tweet does not believe COVID is dangerous.'
df.rename({'text':'premise', 'non_comp':'entailment'}, axis = 1, inplace = True)
df['entailment'].replace({0:1, 1:0}, inplace = True)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\3143349128.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['entailment'].replace({0:1, 1:0}, inplace = True)


### Few Shot

In [9]:
training_args = TrainingArguments(output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    lr_scheduler_type= "linear",
    group_by_length=False,
    learning_rate = 9e-6,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 1, 
    num_train_epochs=5,
    warmup_ratio=0.06,  
    weight_decay=0.01, 
    fp16=True,   
    fp16_full_eval=True,
    eval_strategy="no",
    seed=1,
    save_strategy="no",
    dataloader_num_workers = 12,
)

tokenizer = AutoTokenizer.from_pretrained(modname)

In [10]:
%%time
# Define a function to initialize the modelin the trainer. This will make results reproducible
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modname, num_labels = 2, ignore_mismatched_sizes=True)

# Define the number of samples (shots) and random seeds to use
shots = [10, 25, 50, 100]
seeds = [1,2,3,4,5,6,7,8,9,10]

# Initialize lists to store results
mcc_list = []
acc_list = []
shots_list = []

# Iterate through different shot sizes
for shot in shots:
    # Iterate through different random seeds
    for seed in seeds:
        # Sample training data based on current shot size and seed
        train = df.sample(shot, random_state = seed)
        # Create validation set with remaining instances
        val = df[~df.index.isin(train.index)]
        
        # Create a DatasetDict with train and validation splits
        ds = DatasetDict({'train': Dataset.from_pandas(train, preserve_index=False), 'validation':Dataset.from_pandas(val, preserve_index=False)})
        # Tokenize the dataset
        dstok = ds.map(tokenize_function, batched = True)
        # Rename 'entailment' column to 'label'
        dstok = dstok.rename_columns({'entailment':'label'})
        # Define label mapping
        id2label = {0: "entailment", 1: "not_entailment"}
        
        # Initialize the Trainer
        trainer = Trainer(
            model_init = model_init,
            tokenizer=tokenizer,
            args=training_args,
            train_dataset=dstok['train'],
            eval_dataset=dstok['validation'],
            compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=list(model.config.id2label.values()))
        )
        
        # Train the model
        trainer.train()
        # Make predictions on validation set
        res = trainer.predict(dstok['validation'])
        preds = np.argmax(res.predictions, axis=-1)
        
        # Calculate Matthews Correlation Coefficient
        mcc_res = matthews_corrcoef(val['entailment'], preds)
        mcc_list.append(mcc_res)
        # Calculate Accuracy
        acc_res = accuracy_score(val['entailment'], preds)
        acc_list.append(acc_res)
        # Store the current shot size
        shots_list.append(shot)
    
    # Print progress
    print(shot)

Map: 100%|██████████| 1977/1977 [00:00<00:00, 3333.95 examples/s]
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Aggregate metrics:  {'MCC': 0.708103598451791, 'f1_macro': 0.8511007158453332, 'f1_micro': 0.8897319170460294, 'accuracy_balanced': 0.8329427722482741, 'accuracy': 0.8897319170460294, 'precision_macro': 0.8764991673752834, 'recall_macro': 0.8329427722482741, 'precision_micro': 0.8897319170460294, 'recall_micro': 0.8897319170460294}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\1171338601.py:86: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.8526077097505669, 'recall': 0.7107750472589792, 'f1-score': 0.7752577319587629, 'support': 529.0}, 'not_entailment': {'precision': 0.900390625, 'recall': 0.955110497237569, 'f1-score': 0.9269436997319035, 'support': 1448.0}, 'accuracy': 0.8897319170460294, 'macro avg': {'precision': 0.8764991673752834, 'recall': 0.8329427722482741, 'f1-score': 0.8511007158453332, 'support': 1977.0}, 'weighted avg': {'precision': 0.8876050093363934, 'recall': 0.8897319170460294, 'f1-score': 0.8863560027405067, 'support': 1977.0}} 



...

Map: 100%|██████████| 1887/1887 [00:00<00:00, 3423.18 examples/s]
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Aggregate metrics:  {'MCC': 0.6633062883250996, 'f1_macro': 0.8239313706118685, 'f1_micro': 0.8738738738738738, 'accuracy_balanced': 0.7980748916537717, 'accuracy': 0.8738738738738738, 'precision_macro': 0.8690139998798294, 'recall_macro': 0.7980748916537717, 'precision_micro': 0.8738738738738738, 'recall_micro': 0.8738738738738738}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\1171338601.py:86: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.8609625668449198, 'recall': 0.6338582677165354, 'f1-score': 0.7301587301587301, 'support': 508.0}, 'not_entailment': {'precision': 0.8770654329147389, 'recall': 0.962291515591008, 'f1-score': 0.9177040110650069, 'support': 1379.0}, 'accuracy': 0.8738738738738738, 'macro avg': {'precision': 0.8690139998798294, 'recall': 0.7980748916537717, 'f1-score': 0.8239313706118685, 'support': 1887.0}, 'weighted avg': {'precision': 0.8727303741105692, 'recall': 0.8738738738738738, 'f1-score': 0.8672148734389399, 'support': 1887.0}} 

100
CPU times: total: 41min 8s
Wall time: 3h 44min 56s


In [11]:
res_df = pd.DataFrame({'n':shots_list, 'mcc':mcc_list})
res_df['accuracy'] = acc_list
res_df.groupby('n').mean()

,mcc,accuracy
n,,
10,0.669726,0.869095
25,0.687318,0.877523
50,0.676680,0.876923
100,0.705905,0.887970


# Covid Zero Shot

In [13]:
pipe = pipeline("zero-shot-classification", model = modname, device = 0, batch_size = 32)
res = pipe(list(df['premise'].str.lower()), ['The author of this tweet does not believe COVID is dangerous'], hypothesis_template = '{}.', multi_label = False)
labels = [round(label['scores'][0], 0) for label in res]
df['0_shot'] = labels
df['0_shot'].replace({0:1, 1:0}, inplace = True)

zs_df = pd.DataFrame({'n':0, 'mcc':matthews_corrcoef(df['entailment'], df['0_shot']), 'accuracy':accuracy_score(df['entailment'], df['0_shot'])}, index = [0])
res_df = pd.concat([res_df, zs_df], axis = 0)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_13768\2415661491.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['0_shot'].replace({0:1, 1:0}, inplace = True)


In [14]:
res_df.to_csv('covid_fewshot_large2.csv', index = False)